In [1]:
import os
import sys
import scipy.io as spio
import numpy as np
import plotly.graph_objects as pgo
import re

sys.path.append("../")
from helpers.get_density import get_density
from helpers.get_spins import get_spins
from helpers.downscale import downscale_3d
from plot_multiple_3Ds import plot_multiple_3Ds
from helpers.grids import get_kw_square_nonzero_grid, get_radius_grid, get_space_grid
from loader import load_snaps_at
from multiprocessing import Pool


In [14]:
box_N = 128
box_L = 50.0
box_dx = box_L / box_N

VOL_SCALE_FACTOR = 4
vol_grid_axis = np.linspace((-box_L + box_dx) / 2, (box_L - box_dx) / 2, num=box_N//VOL_SCALE_FACTOR)
(GX, GY, GZ) = [gi.flatten() for gi in np.meshgrid(vol_grid_axis, vol_grid_axis, vol_grid_axis, indexing='ij')]

In [18]:
ld_data = {}
lds = [-13, -10, -7, -4]
for ld in lds:
	# Psis = load_snaps_at(f"../out_remote/2022-11-02/condensation,sigma=0.5,L=100,density=0.0005,lambda={ld}/", [0.05, 0.50, 0.95], box_L, box_N)
	ld_data[ld] = np.genfromtxt(f"../out_remote/2022-11-02/condensation,sigma=0.5,L=100,density=0.0005,lambda={ld}/log.csv", delimiter=',')

In [19]:
fig = pgo.Figure(data=[
	pgo.Scatter(x=ld_data[ld][:, 0], y=ld_data[ld][:, 9], name=f"lambda={ld}")
	for ld in lds
])
fig.update_xaxes(type="log")
fig.update_yaxes(type="log")

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'name': 'lambda=-13',
              'type': 'scatter',
              'x': array([   0.        ,   78.125     ,  156.25      ,  234.375     ,
                           312.5       ,  390.625     ,  468.75      ,  546.875     ,
                           625.        ,  703.125     ,  781.25      ,  859.375     ,
                           937.5       ,  986.20817001,  987.48298488,  988.52766024,
                           989.55756279,  990.57615614,  991.58559463,  992.58785545,
                           993.58478791,  994.57790095,  995.56801765,  996.55542426,
                           997.5402354 ,  998.52302434,  999.50419235, 1000.48373206,
                          1001.46195786, 1002.43929697, 1003.41595394, 1004.39202607,
                          1005.36765285, 1006.34286731, 1007.31756308, 1008.29165106,
                          1009.26494884, 1010.2373212 , 1011.20876665, 1012.17916343,
                          1013.14833565, 1014.11628905, 1015.0831238 , 1016.04881283,
                          1017.01321364, 1017.97636648, 1018.93839572, 1019.89942813,
                          1020.85952799, 1021.8187978 , 1022.77744552, 1023.73556804,
                          1024.6934064 , 1025.65105636, 1026.60861111, 1027.56606764,
                          1028.52341114, 1029.48064305, 1030.43763037, 1031.39417721,
                          1032.35032519, 1033.30618006, 1034.26153741, 1035.2162619 ,
                          1036.17053528, 1037.12442537, 1038.07794218, 1039.03113145,
                          1039.98407682, 1040.93677735, 1041.88930524, 1042.84164094,
                          1043.79370386, 1044.74547838, 1045.69706055, 1046.64828753,
                          1047.59887287, 1048.54906604, 1049.49908369, 1050.44868471,
                          1051.39785002, 1052.34678339, 1053.2955685 , 1054.24422985,
                          1055.19280505, 1056.14129855, 1057.08973588, 1058.0381349 ,
                          1058.98651927, 1059.93489099, 1060.88320049, 1061.83143252,
                          1062.77963667, 1063.72782982, 1064.6759621 , 1065.62401563,
                          1066.57199944, 1067.51995697, 1068.46781532, 1069.41558842,
                          1070.3632956 , 1071.31094906, 1072.25844799, 1073.20582199,
                          1074.1531958 , 1075.10048153, 1076.04760589, 1076.99461999,
                          1077.94159038, 1078.88842557, 1079.83515998, 1080.78181334,
                          1081.7283412 , 1082.67474564, 1083.62107126, 1084.56729314,
                          1085.51340124, 1086.45944454, 1087.40540394, 1088.35129706,
                          1089.29713975, 1090.24293517, 1091.18869038, 1092.13440899,
                          1093.08010305, 1094.02577421, 1094.97145469, 1095.91709727,
                          1096.86268724, 1097.80823421, 1098.75371553, 1099.6990538 ,
                          1100.64428583, 1101.58945057, 1102.53445552, 1103.47929257,
                          1104.42399033, 1105.36849608, 1106.31279007, 1107.25685991,
                          1108.20065072, 1109.14425568, 1110.08765783, 1111.03088114,
                          1111.97398928, 1112.91696314, 1113.85981875, 1114.80262157,
                          1115.74546573, 1116.6883238 , 1117.63112892, 1118.5738741 ,
                          1119.51660582, 1120.45918915, 1121.40158856, 1122.34387907,
                          1123.28609078, 1124.228176  , 1125.17012419, 1126.1119012 ,
                          1127.05362726, 1127.99534646, 1128.93697655, 1129.87853348,
                          1130.82002756, 1131.76144262, 1132.70266609, 1133.64362999,
                          1134.58430213, 1135.52466986, 1136.46473533, 1137.40448729,
                          1138.34395867, 1139.28322772, 1140.22227698, 1141.1610848 ,
                          1142.09965203, 1143.03812284, 1143.97647097, 1144.91462278,
                          1145.85248176, 1146.790230

In [20]:

fig = pgo.Figure(data=[
	pgo.Scatter(x=lds, y=[np.argmax(ld_data[ld][:, 9] > 0.1) for ld in lds], name=f"lambda={ld}")
])
# fig.update_xaxes(type="log")
# fig.update_yaxes(type="log")
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'name': 'lambda=-4', 'type': 'scatter', 'x': [-13, -10, -7, -4], 'y': [13, 23, 29, 44]}],
    'layout': {'template': '...'}
})

In [31]:
def compare_sm():
	lds = [-13, -10]
	sms = [0.3, 0.4, 0.6, 0.7]
	data = {ld: {} for ld in lds}
	for ld in lds:
		for sm in sms:
			# Psis = load_snaps_at(f"../out_remote/2022-11-02/condensation,sigma=0.5,L=100,density=0.0005,lambda={ld}/", [0.05, 0.50, 0.95], box_L, box_N)
			data[ld][sm] = np.genfromtxt(f"../out_remote/2022-11-02/condensation,sigma={sm},L=50,density=0.00016,lambda={ld}/log.csv", delimiter=',')
	#(print(f"{ld} {sm}"))
	data = [
		pgo.Scatter(x=data[ld][sm][:, 0], y=data[ld][sm][:, 9], name=f"lambda={ld},sigma={sm}")
		for ld in lds
		for sm in sms
	]
	fig = pgo.Figure(data=data)
	fig.update_xaxes(type="log")
	fig.update_yaxes(type="log")
	return fig
compare_sm()
	

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'name': 'lambda=-13,sigma=0.3',
              'type': 'scatter',
              'x': array([   0.     ,   19.53125,   39.0625 ,   58.59375,   78.125  ,   97.65625,
                           117.1875 ,  136.71875,  156.25   ,  175.78125,  195.3125 ,  214.84375,
                           234.375  ,  253.90625,  273.4375 ,  292.96875,  312.5    ,  332.03125,
                           351.5625 ,  371.09375,  390.625  ,  410.15625,  429.6875 ,  449.21875,
                           468.75   ,  488.28125,  507.8125 ,  527.34375,  546.875  ,  566.40625,
                           585.9375 ,  605.46875,  625.     ,  644.53125,  664.0625 ,  683.59375,
                           703.125  ,  722.65625,  742.1875 ,  761.71875,  781.25   ,  800.78125,
                           820.3125 ,  839.84375,  859.375  ,  878.90625,  898.4375 ,  917.96875,
                           937.5    ,  957.03125,  976.5625 ,  996.09375, 1015.625  , 1035.15625,
                          1054.6875 , 1074.21875, 1093.75   , 1113.28125, 1132.8125 , 1152.34375,
                          1171.875  , 1191.40625, 1210.9375 , 1230.46875, 1250.     , 1269.53125,
                          1289.0625 , 1308.59375, 1328.125  , 1347.65625, 1367.1875 , 1386.71875,
                          1406.25   , 1425.78125, 1445.3125 , 1464.84375, 1484.375  , 1503.90625,
                          1523.4375 , 1542.96875, 1562.5    , 1582.03125, 1601.5625 , 1621.09375,
                          1640.625  , 1660.15625, 1679.6875 , 1699.21875, 1718.75   , 1738.28125,
                          1757.8125 , 1777.34375, 1796.875  , 1816.40625, 1835.9375 , 1855.46875,
                          1875.     , 1894.53125, 1914.0625 , 1933.59375, 1953.125  , 1972.65625,
                          1992.1875 , 2011.71875, 2031.25   , 2050.78125, 2070.3125 , 2089.84375,
                          2109.375  , 2128.90625, 2148.4375 , 2167.96875, 2187.5    , 2207.03125,
                          2226.5625 , 2246.09375, 2265.625  , 2285.15625, 2304.6875 , 2324.21875,
                          2343.75   , 2363.28125, 2382.8125 , 2402.34375, 2421.875  , 2441.40625,
                          2460.9375 , 2480.46875, 2500.     , 2519.53125, 2539.0625 , 2558.59375,
                          2578.125  , 2597.65625, 2617.1875 , 2636.71875, 2656.25   , 2675.78125,
                          2695.3125 , 2714.84375, 2734.375  , 2753.90625, 2773.4375 , 2792.96875,
                          2812.5    , 2832.03125, 2851.5625 , 2871.09375, 2890.625  , 2910.15625,
                          2929.6875 , 2949.21875, 2968.75   , 2988.28125, 3007.8125 , 3027.34375,
                          3046.875  , 3066.40625, 3085.9375 , 3105.46875, 3125.     , 3144.53125,
                          3164.0625 , 3183.59375, 3203.125  , 3222.65625, 3242.1875 , 3261.71875,
                          3281.25   , 3300.78125, 3320.3125 , 3339.84375, 3359.375  , 3378.90625,
                          3398.4375 , 3417.96875, 3437.5    , 3457.03125, 3476.5625 , 3496.09375,
                          3515.625  , 3535.15625, 3554.6875 , 3574.21875, 3593.75   , 3613.28125,
                          3632.8125 , 3652.34375]),
              'y': array([0.00176208, 0.00200045, 0.00158236, 0.00161745, 0.00219086, 0.0016501 ,
                          0.00204231, 0.00166235, 0.00149796, 0.00151835, 0.0016429 , 0.00143334,
                          0.00196002, 0.00210599, 0.0016399 , 0.00187145, 0.00155893, 0.00135864,
                          0.00231717, 0.00181291, 0.00169189, 0.00156977, 0.00185992, 0.00144149,
                          0.00151777, 0.00149818, 0.00158794, 0.00201275, 0.00178953, 0.0014693 ,
                          0.00160567, 0.00149244, 0.00216318, 0.00217997, 0.00168638, 0.00160142,
                          0.00188714, 0.00142892, 0.00140797, 0.00216656, 0.00166107, 0.00140928,
                          0.00186061, 0.00169941, 0.00162183, 0.00170746, 0.00201075, 0.00182237,
                          0.00157721, 

In [100]:
Space3 = get_space_grid(box_N, box_dx)
def plot_rho_over_r(Psi3):
    Rho = get_density(Psi3)
    Rho_f = Rho.flatten()
    densest_point_idx = np.unravel_index(np.argmax(Rho_f), shape=Rho.shape)
    densest_point = tuple([sp[densest_point_idx] for sp in Space3])
    radius = get_radius_grid(box_N, box_dx, densest_point)
    radius_f = radius.flatten()
    sorter = np.argsort(radius_f)
    radius_s = radius_f[sorter]
    Rho_s = Rho_f[sorter]
    counts_we, edges = np.histogram(
        radius_s, weights=Rho_s, bins=np.linspace(box_dx, box_L, 400))
    counts_uw, _ = np.histogram(radius_s, bins=np.linspace(box_dx, box_L, 400))
    y = counts_we/np.maximum(counts_uw, 1)
    x = edges
    return pgo.Scatter(x=x, y=y, mode='markers')


In [116]:
def plot_est_profile(Psi3):
    Rho = get_density(Psi3)
    Rho_f = Rho.flatten()
    # densest_point_idx = np.unravel_index(np.argmax(Rho_f), shape=Rho.shape)
    # densest_point = tuple([sp[densest_point_idx] for sp in Space3])
    # amp = np.sqrt(np.average(np.sort(Rho_f)[-8:]))
    amp = np.sqrt(1)
    print(amp)
    x = np.linspace(box_dx, box_L, 400)
    y = amp**2 * 0.998309 / (1. + 0.037653 * amp * np.square(x))**8
    return pgo.Scatter(x=x, y=y, mode='lines')

In [117]:
Psis[-1].shape

(3, 128, 128, 128)

In [118]:
fig = pgo.Figure()
fig.add_trace(plot_rho_over_r(Psis[-1]))
fig.add_trace(plot_est_profile(Psis[-1]))
fig.update_yaxes(type="log")
fig.update_xaxes(type="log")
pass
fig.write_html("2022-10-23_Rho_over_r.html")

1.0
